In [5]:
import torch

In [6]:
with open('../data/tinyshakespeare/input.txt','r',encoding='utf-8') as f:
    text = f.read()

In [7]:
print('Total rows in text',len(text))

Total rows in text 1115394


In [8]:
text[:1000]

"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger 

In [9]:
# build vocabulary table
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f'chars are: {''.join(chars)}')
print(f'vocab_size is: {vocab_size}')

chars are: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab_size is: 65


In [10]:
# build tokenizer
itos ={i:ch for i,ch in enumerate(chars)}
stoi ={ch:i for i,ch in enumerate(chars)}

encode = lambda s:[stoi[c] for c in s]
decode = lambda l:''.join([itos[i] for i in l])

In [11]:
print('hi there:',decode(encode('hi there')))

hi there: hi there


In [12]:
data = torch.tensor(encode(text),dtype=torch.long)
print(f'data shape and dtype are: {data.shape} {data.dtype}')
print(data[:10])

data shape and dtype are: torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])


In [13]:
# 将数据分割成训练集和测试集
n = int(len(data) * 0.9)
train_data = data[:n]
val_data = data[n:]

In [14]:
# 上下文长度
block_size = 8
# 9个训练数据实际包含了8组训练数据
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [15]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'when input sequence is {context}, the target is {target}')

when input sequence is tensor([18]), the target is 47
when input sequence is tensor([18, 47]), the target is 56
when input sequence is tensor([18, 47, 56]), the target is 57
when input sequence is tensor([18, 47, 56, 57]), the target is 58
when input sequence is tensor([18, 47, 56, 57, 58]), the target is 1
when input sequence is tensor([18, 47, 56, 57, 58,  1]), the target is 15
when input sequence is tensor([18, 47, 56, 57, 58,  1, 15]), the target is 47
when input sequence is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is 58


In [16]:
# build batch，通过batch可以加速训练过程
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [21]:
x,y = get_batch('train')
print(f'x is:{x}')
print(f'y is:{y}')

print('-'*30)
for b in range(batch_size):
    for t in range(block_size):
        context = x[b][:t+1]
        target = y[b][t]
        print(f'when input sequence is {context}, the target is {target}')

x is:tensor([[ 1, 39, 52, 42,  1, 45, 43, 50],
        [ 1, 58, 46, 39, 58,  1, 42, 53],
        [ 1, 61, 53, 59, 50, 42,  1, 21],
        [59, 57, 40, 39, 52, 42,  1, 40]])
y is:tensor([[39, 52, 42,  1, 45, 43, 50, 42],
        [58, 46, 39, 58,  1, 42, 53,  1],
        [61, 53, 59, 50, 42,  1, 21,  1],
        [57, 40, 39, 52, 42,  1, 40, 47]])
------------------------------
when input sequence is tensor([1]), the target is 39
when input sequence is tensor([ 1, 39]), the target is 52
when input sequence is tensor([ 1, 39, 52]), the target is 42
when input sequence is tensor([ 1, 39, 52, 42]), the target is 1
when input sequence is tensor([ 1, 39, 52, 42,  1]), the target is 45
when input sequence is tensor([ 1, 39, 52, 42,  1, 45]), the target is 43
when input sequence is tensor([ 1, 39, 52, 42,  1, 45, 43]), the target is 50
when input sequence is tensor([ 1, 39, 52, 42,  1, 45, 43, 50]), the target is 42
when input sequence is tensor([1]), the target is 58
when input sequence is ten